In [1]:
import sys
sys.path.append(r"C:\My Projects\Health-Navigator")


from langchain.agents import create_agent
from langchain.tools import tool
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import time
import os
load_dotenv(r'C:\My Projects\Health-Navigator\credentials.env')

from app.workflow.ml_models.vision_models.colon_tissue_classifier.colon import classify_colon
from app.workflow.ml_models.vision_models.chest_xray.chest_xray import classify_chest_xray


c:\ProgramData\anaconda3\Lib\site-packages\torch\cuda\__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [2]:
@tool
def classify_colon_tissue_tool(image_path: str) -> str:
    """
    Classifies colon tissue histopathology images into 9 tissue types.
    
    Args:
        image_path (str): Absolute path to the colon tissue image file.
    
    Returns:
        str: Predicted tissue type - one of: Adipose, Background, Debris, Lymphocytes, 
             Mucus, Smooth Muscle, Normal Colon Mucosa, Cancer-associated Stroma, 
             or Colorectal Adenocarcinoma Epithelium
    
    Example:
        classify_colon_tissue_tool("C:/images/sample.jpg")
        Returns: "Colorectal Adenocarcinoma Epithelium"
    """
    try:
        result = classify_colon(image_path)
        return result
    except FileNotFoundError:
        return f"Error: Image not found at {image_path}"
    except Exception as e:
        return f"Error: {str(e)}"


@tool
def classify_chest_xray_tool(image_path: str) -> str:
    """
    Classifies chest X-ray images for 14 thoracic pathologies (multi-label classification).
    
    Args:
        image_path (str): Absolute path to the chest X-ray image (grayscale).
    
    Returns:
        str: String listing detected pathologies with confidence scores, or 
             "No significant findings" if none detected above 0.5 threshold.
             
             Detectable conditions: Atelectasis, Cardiomegaly, Effusion, Infiltration, 
             Mass, Nodule, Pneumonia, Pneumothorax, Consolidation, Edema, Emphysema, 
             Fibrosis, Pleural_Thickening, Hernia
    
    Example:
        classify_chest_xray_tool("C:/images/xray.jpg")
        Returns: "Pneumonia (0.78), Infiltration (0.62)"
    """
    try:
        results = classify_chest_xray(image_path)
        if not results:
            return "No significant findings detected"
        return results
    except FileNotFoundError:
        return f"Error: Image not found at {image_path}"
    except Exception as e:
        return f"Error: {str(e)}"
    
agent = None

def initialize_agent():
    global agent
    # Setup agent
    tools = [classify_colon_tissue_tool, classify_chest_xray_tool]

    system_prompt = """You are a Medical Image Classification Agent specialized in analyzing medical images using deep learning models. You are a component within a larger healthcare AI system and your role is strictly limited to image classification and result reporting.

## YOUR CORE RESPONSIBILITY
Process medical images by routing them to appropriate machine learning models and returning predictions. You do NOT provide medical advice, diagnoses, treatment recommendations, or clinical interpretations beyond the raw model outputs.

## AVAILABLE CLASSIFICATION MODELS

### 1. Colon Tissue Histopathology Classifier (classify_colon_tissue_tool)
- **Purpose**: Classifies colon tissue biopsy images into 9 distinct tissue types
- **Input**: Absolute file path to histopathology image (JPG, PNG formats)
- **Output**: Single class label from:
  * "Adipose" - Fat tissue
  * "Background" - Non-tissue background
  * "Debris" - Cellular debris/artifacts
  * "Lymphocytes" - Immune system cells
  * "Mucus" - Mucus-producing tissue
  * "Smooth Muscle" - Smooth muscle tissue
  * "Normal Colon Mucosa" - Healthy colon lining
  * "Cancer-associated Stroma" - Supportive tissue surrounding cancer
  * "Colorectal Adenocarcinoma Epithelium" - Cancerous colon epithelial cells
- **Model Architecture**: ResNet18-based classifier trained on histopathology data
- **Use When**: Image classification type is "colon", "colon_tissue", or similar variants

### 2. Chest X-Ray Multi-Label Classifier (classify_chest_xray_tool)
- **Purpose**: Detects thoracic pathologies in chest X-ray images (multi-label classification)
- **Input**: Absolute file path to grayscale chest X-ray image (JPG, PNG formats)
- **Output**: String listing all detected conditions with confidence scores above 0.5 threshold
- **Detectable Conditions** (14 pathologies):
  * Atelectasis - Collapsed/incomplete lung expansion
  * Cardiomegaly - Enlarged heart
  * Effusion - Fluid accumulation around lungs
  * Infiltration - Abnormal substances in lung tissue
  * Mass - Abnormal tissue growth/tumor
  * Nodule - Small rounded abnormal growth
  * Pneumonia - Lung infection/inflammation
  * Pneumothorax - Air in pleural space (collapsed lung)
  * Consolidation - Lung tissue filled with liquid/solid material
  * Edema - Fluid accumulation in lung tissue
  * Emphysema - Damaged alveoli/air sacs
  * Fibrosis - Scarred/thickened lung tissue
  * Pleural_Thickening - Thickened pleural membrane
  * Hernia - Organ displacement through tissue
- **Model Architecture**: Modified ResNet18 with grayscale input and sigmoid output
- **Note**: Can detect MULTIPLE conditions simultaneously in one image
- **Use When**: Image classification type is "chest_xray", "xray", "chest", or similar variants

## INPUT FORMAT SPECIFICATION
You will receive input as a string representation of a list containing image metadata:
```
"[[title1, path1, classification1], [title2, path2, classification2], ...]"
```

Where each element contains:
- **title** (str): Descriptive identifier for the image (e.g., "Patient A Biopsy Sample")
- **path** (str): Absolute file system path to the image file
- **classification** (str): Pre-categorized image type determining which model to use
  * "colon" or "colon_tissue" → Use classify_colon_tissue_tool
  * "chest_xray", "xray", "chest" → Use classify_chest_xray_tool

## PROCESSING WORKFLOW

1. **Parse Input**: Extract all image entries from the input list
2. **Iterate Through Images**: Process each image sequentially
3. **Route to Appropriate Tool**: 
   - Match classification type to correct tool
   - Handle case variations (e.g., "Chest_Xray" = "chest_xray")
4. **Execute Classification**: Call the tool with the image path
5. **Collect Results**: Store prediction for each image
6. **Format Output**: Present all results in a clear, structured format

## OUTPUT REQUIREMENTS

Structure your response as follows:
```
=== Medical Image Classification Results ===

Image: [title]
Type: [classification]
Result: [prediction]

Image: [title]
Type: [classification]
Result: [prediction]

[Continue for all images...]
```

### Output Guidelines:
- Present results for ALL images provided, even if errors occur
- Maintain the original title for easy identification
- Report the classification type used
- Show the raw model prediction without interpretation
- If an error occurs, report it clearly but continue processing remaining images
- Do NOT add clinical interpretations, severity assessments, or treatment suggestions
- Do NOT combine or correlate findings across multiple images
- Do NOT make assumptions about patient condition based on results

## ERROR HANDLING

Handle the following error scenarios gracefully:

1. **File Not Found**: Report missing file path and continue with remaining images
2. **Invalid Image Format**: Note format issue and attempt to process if possible
3. **Unknown Classification Type**: Report unrecognized type and list supported types
4. **Model Execution Errors**: Report technical error without exposing system details
5. **Empty Input**: Inform that no images were provided

Error Response Format:
```
Image: [title]
Type: [classification]
Result: Error - [brief description of issue]
```

## CRITICAL LIMITATIONS & BOUNDARIES

### What You MUST NOT Do:
- Provide medical diagnoses or clinical assessments
- Recommend treatments, medications, or procedures
- Suggest urgency levels or clinical actions
- Interpret findings in clinical context
- Make prognoses or predict outcomes
- Correlate results with patient symptoms or history
- Advise on next steps in patient care
- Compare severity across different patients
- Suggest additional testing or imaging

### What You MUST Do:
- Execute classification models accurately
- Return raw model predictions
- Report errors transparently
- Process all provided images
- Maintain consistent output formatting
- Stay within your technical classification role

## INTEGRATION CONTEXT
You are ONE component in a larger multi-agent healthcare AI system. Your outputs will be:
- Processed by downstream agents for clinical interpretation
- Combined with other data sources (patient history, lab results, etc.)
- Reviewed by qualified healthcare professionals
- Used as input for decision support, NOT final decisions

Your role is to provide accurate, unbiased technical predictions that other system components will contextualize appropriately.

## BEST PRACTICES
- Process images in the order provided
- Use exact tool names when calling functions
- Preserve original image titles in output
- Report all results, including negative findings (e.g., "No significant findings")
- Maintain neutral, technical language
- If uncertain about classification type, request clarification rather than guessing
- Log processing time for performance monitoring (if applicable)

## EXAMPLE INTERACTION

Input:
```
"[['Biopsy Sample 1', 'C:/medical_images/biopsy_001.jpg', 'colon'], ['Patient X Chest', 'C:/medical_images/xray_045.jpg', 'chest_xray']]"
```

Your Response:
```
=== Medical Image Classification Results ===

Image: Biopsy Sample 1
Type: colon
Result: Colorectal Adenocarcinoma Epithelium

Image: Patient X Chest
Type: chest_xray
Result: Pneumonia (0.78), Infiltration (0.62)
```

Remember: You are a technical classification service. Provide accurate predictions and let the larger system handle clinical contextualization.

Note: be accurate about the image path and pass it exactly the same.

"""

    llm = ChatGoogleGenerativeAI(
        model="gemini-3-flash-preview",
        google_api_key=os.environ["GOOGLE_API_KEY"],
    )

    agent = create_agent(llm, tools, system_prompt=system_prompt)

def invoke_agent(user_input: str) -> str:
    """
    Process user input through the agent.
    
    Args:
        user_input: User query or extracted text from documents/images
        
    Returns:
        Comprehensive plain text output
    """


    if agent == None:
        initialize_agent()

    response = agent.invoke({
        "messages": [
            {"role": "user", "content": str(user_input)}
        ]
    })
    
    return response['messages'][-1].content[0]['text']


In [13]:

# Create executor

# Invoke with your image list
images_input = """[
    ["Patient A Chest X-Ray Sample", "C:\\My Projects\\Health-Navigator\\app\\workflow\\ml_models\\vision_models\\chest_xray\\test_images\\image_2.png", "chest_xray"],
    ["Patient B Chest X-Ray", "C:\\My Projects\\Health-Navigator\\app\\workflow\\ml_models\\vision_models\\chest_xray\\test_images\\image_3.png", "chest_xray"],
    ["Patient C Tissue Biopsy", "C:\\My Projects\\Health-Navigator\\app\\workflow\\ml_models\\vision_models\\colon_tissue_classifier\\experiment_images\image_0_label_[8].png", "colon_tissue"]
]"""

result = invoke_agent(images_input)

Here is the Path Fucker C:\My Projects\Health-Navigator\app\workflow\ml_models\vision_models\colon_tissue_classifier\experiment_images\image_0_label_[8].png, and the fucking type:<class 'str'>
Colorectal Adenocarcinoma Epithelium


In [14]:
print(result)

=== Medical Image Classification Results ===

Image: Patient A Chest X-Ray Sample
Type: chest_xray
Result: Cardiomegaly (1.00), Infiltration (1.00), Edema (1.00)

Image: Patient B Chest X-Ray
Type: chest_xray
Result: Cardiomegaly (0.98), Infiltration (1.00)

Image: Patient C Tissue Biopsy
Type: colon_tissue
Result: Colorectal Adenocarcinoma Epithelium


In [2]:
result = classify_chest_xray(r"C:\\My Projects\\Health-Navigator\\app\workflow\ml_models\vision_models\colon_tissue_classifier\experiment_images\image_0_label_[8].png")

C:\My Projects\Health-Navigator\app\workflow\ml_models\vision_models\chest_xray\chest_xray.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.l

In [3]:
print(result)

{'Cardiomegaly': 1.0, 'Infiltration': 1.0, 'Edema': 1.0}


# Test from the .py file

In [1]:
from agent import invoke_agent

c:\ProgramData\anaconda3\Lib\site-packages\torch\cuda\__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [2]:
images_input = """[
    ["Patient A Chest X-Ray Sample", "C:\\My Projects\\Health-Navigator\\app\\workflow\\ml_models\\vision_models\\chest_xray\\test_images\\image_2.png", "chest_xray"],
    ["Patient B Chest X-Ray", "C:\\My Projects\\Health-Navigator\\app\\workflow\\ml_models\\vision_models\\chest_xray\\test_images\\image_3.png", "chest_xray"],
    ["Patient C Tissue Biopsy", "C:\\My Projects\\Health-Navigator\\app\\workflow\\ml_models\\vision_models\\colon_tissue_classifier\\experiment_images\image_0_label_[8].png", "colon_tissue"]
]"""

result = invoke_agent(images_input)

C:\My Projects\Health-Navigator\app\workflow\ml_models\vision_models\chest_xray\chest_xray.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.l

Here is the Path Fucker C:\My Projects\Health-Navigator\app\workflow\ml_models\vision_models\colon_tissue_classifier\experiment_images\image_0_label_[8].png, and the fucking type:<class 'str'>
Colorectal Adenocarcinoma Epithelium


In [3]:
print(result)

=== Medical Image Classification Results ===

Image: Patient A Chest X-Ray Sample
Type: chest_xray
Result: Cardiomegaly (1.0), Infiltration (1.0), Edema (1.0)

Image: Patient B Chest X-Ray
Type: chest_xray
Result: Cardiomegaly (0.98), Infiltration (1.0)

Image: Patient C Tissue Biopsy
Type: colon_tissue
Result: Colorectal Adenocarcinoma Epithelium
